In [2]:
'''
계산한 단기 지표들을 데이터베이스에서 꺼냄
'''
import pandas as pd
import sqlite3

db_path = 'Database/S&P500.db'
conn = sqlite3.connect(db_path)

query = "SELECT gics_sector FROM stock;"
stocks = pd.read_sql_query(query, conn)
sectors = stocks['gics_sector'].unique()
print(sectors)

query = "SELECT * FROM daily_stock_indicator;"
daily_stock_indicator = pd.read_sql_query(query, conn)
display(daily_stock_indicator.head())

conn.close()

['Industrials' 'Health Care' 'Information Technology' 'Utilities'
 'Financials' 'Materials' 'Consumer Discretionary' 'Real Estate'
 'Communication Services' 'Consumer Staples' 'Energy']


,symbol,gics_sector,date,adj_close,close,high,low,open,volume,sma_5,...,change_19td,change_20td,change_21td,change_22td,change_23td,change_24td,change_25td,change_26td,change_27td,change_28td
0,MMM,Industrials,2010-01-04 00:00:00,43.783875,69.414719,69.774246,69.122070,69.473244,3640265,NaN,...,-3.107693,-3.119739,-1.830886,-4.589262,-5.396289,-6.612865,-5.179480,-5.299931,-3.312456,-4.625400
1,MMM,Industrials,2010-01-05 00:00:00,43.509628,68.979935,69.590302,68.311035,69.230766,3405012,NaN,...,-2.509098,-1.212121,-3.987883,-4.799998,-6.024241,-4.581822,-4.703032,-2.703029,-4.024250,-2.460609
2,MMM,Industrials,2010-01-06 00:00:00,44.126682,69.958191,70.735786,69.824417,70.133781,6301126,NaN,...,-2.593516,-5.330463,-6.131221,-7.338346,-5.916096,-6.035611,-4.063576,-5.366321,-3.824546,-4.218948
3,MMM,Industrials,2010-01-07 00:00:00,44.158318,70.008362,70.033447,68.662209,69.665550,5346240,NaN,...,-5.398307,-6.198491,-7.404751,-5.983521,-6.102950,-4.132328,-5.434139,-3.893469,-4.287589,-3.129106
4,MMM,Industrials,2010-01-08 00:00:00,44.469475,70.501671,70.501671,69.648827,69.974915,4073337,69.772575,...,-6.854832,-8.052651,-6.641366,-6.759960,-4.803126,-6.095828,-4.565938,-4.957301,-3.806924,-3.320677


In [29]:
'''
데이터 전처리
'''
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

data_all = daily_stock_indicator.copy()


# 결측치가 존재하는 행 제거 (대략 1.8%)
data_all = data_all.dropna()


# 날짜 컬럼 타입변경
data_all['date'] = pd.to_datetime(data_all['date'])


# n일 뒤 변화율만 남김
after_td = 3
data_all = data_all.drop(columns=[f'change_{i}td' for i in range(1, 7 * 4 + 1) if i != after_td])
data_all = data_all.rename(columns={f'change_{after_td}td':'change'})


# 데이터 정규화
mmscaler = MinMaxScaler(feature_range=(0, 1))
data_all = data_all.set_index(['date', 'symbol', 'gics_sector'])
tmp1 = data_all.loc[ : ,  :'three_white_soldiers']
tmp2 = data_all.iloc[ : , -1]
data_all = pd.concat([pd.DataFrame(data=mmscaler.fit_transform(tmp1), index=data_all.index, columns=data_all.loc[ : , :'three_white_soldiers'].columns), tmp2], axis=1)
data_all = data_all.reset_index()


display(data_all.head())

,date,symbol,gics_sector,adj_close,close,high,low,open,volume,sma_5,sma_10,sma_20,ema_5,ema_10,ema_20,macd,macd_signal,macd_diff,adx,adx_pos,adx_neg,parabolic_sar,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,rsi_14,stochastic_k,stochastic_d,tsi,roc_12,cmo,williams_%r,cci_10,cci_20,bollinger_upper,bollinger_middle,bollinger_lower,atr_14,keltner_hband,keltner_lband,donchian_hband,donchian_lband,obv,mfi_14,volume_oscillator,adl,vwap_14,pivot_point_1,pivot_point_2,doji,hammer,inverted_hammer,shooting_star,morning_star,triple_top,triple_bottom,red_marubozu,green_marubozu,three_black_crows,three_white_soldiers,change
0,2010-02-26,MMM,Industrials,0.004267,0.006730,0.006718,0.006731,0.006703,0.001044,0.006798,0.006901,0.006885,0.006807,0.006861,0.006951,0.615300,0.577717,0.639274,0.178453,0.186587,0.223297,0.006834,0.006933,0.007334,0.006995,0.006873,0.471976,0.643016,0.606799,0.427517,0.423592,0.573516,0.643016,0.440114,0.499404,0.006839,0.006885,0.010812,0.003941,0.006873,0.006888,0.006845,0.006894,0.089873,0.489782,0.172736,0.104758,0.006855,0.006726,0.006720,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.060509
1,2010-03-01,MMM,Industrials,0.004290,0.006767,0.006770,0.006838,0.006773,0.000824,0.006796,0.006913,0.006885,0.006821,0.006868,0.006952,0.615381,0.577786,0.639333,0.162930,0.210339,0.214227,0.006828,0.006896,0.007334,0.006921,0.006873,0.496328,0.694299,0.629807,0.434668,0.426853,0.647059,0.694299,0.560307,0.550588,0.006840,0.006885,0.010812,0.003814,0.006868,0.006892,0.006835,0.006894,0.089897,0.557064,0.172738,0.104750,0.006874,0.006791,0.006786,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.546047
2,2010-03-02,MMM,Industrials,0.004319,0.006813,0.006806,0.006857,0.006787,0.001090,0.006821,0.006918,0.006888,0.006845,0.006882,0.006958,0.615502,0.577872,0.639462,0.150690,0.223969,0.205241,0.006823,0.006896,0.007334,0.006921,0.006873,0.526425,0.832447,0.723254,0.445031,0.420000,0.623844,0.832447,0.616667,0.572548,0.006848,0.006888,0.010810,0.003696,0.006870,0.006895,0.006835,0.006894,0.089928,0.550195,0.172743,0.104766,0.006889,0.006825,0.006816,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.614687
3,2010-03-03,MMM,Industrials,0.004312,0.006802,0.006831,0.006881,0.006823,0.000924,0.006836,0.006926,0.006891,0.006858,0.006891,0.006962,0.615589,0.577961,0.639496,0.142067,0.230205,0.196152,0.006577,0.006895,0.007334,0.006918,0.006873,0.518557,0.794038,0.773595,0.452690,0.425916,0.621926,0.794038,0.619700,0.575891,0.006854,0.006891,0.010809,0.003590,0.006871,0.006901,0.006835,0.006894,0.089902,0.602315,0.172742,0.104743,0.006901,0.006838,0.006834,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.382710
4,2010-03-04,MMM,Industrials,0.004313,0.006804,0.006804,0.006856,0.006827,0.000849,0.006858,0.006925,0.006889,0.006867,0.006899,0.006966,0.615662,0.578051,0.639496,0.130977,0.220152,0.203553,0.006582,0.006895,0.007334,0.006918,0.006873,0.519749,0.799458,0.808648,0.459234,0.418130,0.546125,0.799458,0.577113,0.564086,0.006847,0.006889,0.010812,0.003485,0.006867,0.006905,0.006835,0.006894,0.089926,0.522508,0.172742,0.104747,0.006908,0.006821,0.006823,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.197233


In [31]:
'''
데이터 선택
'''
# all, sector
selector = 'sector'

dps = {} # data per sector
if selector == 'all':
    dps['all'] = data_all.copy()
elif selector == 'sector':
    for sector in sectors:
        dps[sector] = data_all.loc[data_all['gics_sector'] == sector].copy()

In [ ]:
'''
모델링 및 검증 : LSTM (Long Short-Term Memory)
'''
import time

import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt


accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for sector, data in dps.items():    
    if sector == 'Industrials':
        # 업종 컬럼 삭제
        data = data.drop(columns=['gics_sector'])


        # 종목 컬럼 삭제 or 종목이름 원핫인코딩
        data = data.drop(columns=['symbol'])
        
        

        # 데이터셋 준비
        # 문제점 : 지금 지난 60일 치의 
        # def create_dataset(data, look_back):
        #     X, Y = [], []
        #     for i in range(len(data) - look_back - 1):
        #         X.append(data.iloc[i:(i + look_back), :-1])
        #         Y.append(data.iloc[i + look_back, -1])
        #     return np.array(X), np.array(Y)
        # look_back = 60
        # future_days = 3
        # x_data, y_data = create_dataset(data, look_back)
        

        # 학습, 테스트 데이터 분할 (8:2)
        train_size  = int(len(x_data) * 0.8)
        x_train, x_test = x_data.iloc[ : ,  :train_size], x_data.iloc[ : , train_size: ]
        y_train, y_test = y_data.iloc[ : ,  :train_size], y_data.iloc[ : , train_size: ]
        display(x_train)

        start = time.time()
        
        
        # 데이터 형태 변환 (LSTM 입력 형태)
        # x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1]))
        
        # 모델 학습
        

        
        end = time.time()
        print(f"Learning Time : {end - start:.5f} sec")

    #     # 점수화
    #     acs = accuracy_score(y_test_c, pred_c)
    #     prs = precision_score(y_test_c, pred_c)
    #     res = recall_score(y_test_c, pred_c)
    #     f1s = f1_score(y_test_c, pred_c)
    #     print('SECTOR : ', sector)
    #     print('accuracy_score : ', acs)
    #     print('precision_score : ', prs)
    #     print('recall_score : ', res)
    #     print('f1_score : ', f1s)
    #     accuracy_scores.append(acs)
    #     precision_scores.append(prs)
    #     recall_scores.append(res)
    #     f1_scores.append(f1s)
    #     print()

    #     print("Train Values : ")
    #     display(y_train_c.value_counts())
    #     print()

    #     # print("Validation Values : ")
    #     # display(y_val_c.value_counts())
    #     # print()

    #     print("Test Values : ")
    #     display(y_test_c.value_counts())
    #     print()

    #     print("Predict Values : ")
    #     display(pd.DataFrame({'predict' : pred_c}).value_counts())
    #     print()

# print('accuracy_score mean : ', sum(accuracy_scores) / len(accuracy_scores))
# print('precision_score mean : ', sum(precision_scores) / len(precision_scores))
# print('recall_score mean : ', sum(recall_scores) / len(recall_scores))
# print('f1_score mean : ', sum(f1_scores) / len(f1_scores))

: 

In [51]:
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
tmp = data.head(100).copy()
tmp = tmp.set_index('date')
mmscaler = MinMaxScaler(feature_range=(0, 1))
display(tmp)
display(pd.DataFrame(data=mmscaler.fit_transform(tmp), index=tmp.index, columns=tmp.columns))

,adj_close,close,high,low,open,volume,sma_5,sma_10,sma_20,ema_5,ema_10,ema_20,macd,macd_signal,macd_diff,adx,adx_pos,adx_neg,parabolic_sar,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,rsi_14,stochastic_k,stochastic_d,tsi,roc_12,cmo,williams_%r,cci_10,cci_20,bollinger_upper,bollinger_middle,bollinger_lower,atr_14,keltner_hband,keltner_lband,donchian_hband,donchian_lband,obv,mfi_14,volume_oscillator,adl,vwap_14,pivot_point_1,pivot_point_2,doji,hammer,inverted_hammer,shooting_star,morning_star,triple_top,triple_bottom,red_marubozu,green_marubozu,three_black_crows,three_white_soldiers,change_1td,change_2td,change_3td,change_4td,change_5td,change_6td,change_7td,change_8td,change_9td,change_10td,change_11td,change_12td,change_13td,change_14td,change_15td,change_16td,change_17td,change_18td,change_19td,change_20td,change_21td,change_22td,change_23td,change_24td,change_25td,change_26td,change_27td,change_28td
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-02-26,42.547859,67.015053,67.165550,66.137123,66.672241,3856382,66.948160,67.121237,66.827759,66.979710,67.012310,67.226465,-0.377006,-0.516435,0.139429,21.572467,18.679348,22.463317,68.309032,67.359951,67.901337,67.663044,67.056858,47.143716,64.301582,60.679939,-14.663507,1.816571,14.703179,-35.698418,-39.924183,-0.794047,68.513668,66.827759,65.141850,1.219982,67.934505,65.625974,68.461540,64.590302,-32214619,48.978189,-269299.918633,1.626729e+07,66.604719,66.772575,66.747492,0,0,0,0,0,1,1,0,0,0,1,0.536488,1.222706,1.060509,1.085465,2.857136,1.447278,2.295693,1.759194,1.384902,1.534621,1.384902,1.883969,2.532743,4.391757,2.258260,2.108541,2.744849,1.821581,0.973167,1.060509,1.509666,5.152829,4.266993,4.616341,5.040542,5.240171,4.379291,3.493444
2010-03-01,42.776112,67.374580,67.683945,67.182274,67.366219,3043940,66.931438,67.238294,66.831522,67.111333,67.078177,67.240572,-0.321912,-0.477530,0.155618,20.117116,21.048366,21.558608,68.257902,67.002510,67.901337,66.948162,67.056858,49.572476,69.429939,62.980713,-13.238878,2.493001,29.411713,-30.570061,40.204782,67.450540,68.521953,66.831522,65.141090,1.180618,67.879181,65.662624,68.361206,64.590302,-29170679,55.706407,-252969.551644,1.555703e+07,66.783803,67.413600,67.401754,1,0,0,0,0,1,0,0,0,0,1,0.682556,0.521225,0.546047,2.308264,0.905930,1.749817,1.216181,0.843887,0.992807,0.843887,1.340291,1.985603,3.834696,1.712585,1.563665,2.196577,1.278236,0.434349,0.521225,0.967985,4.591707,3.710598,4.058082,4.480020,4.678583,3.822297,2.941177,3.611322
2010-03-02,43.068089,67.834450,68.043480,67.374580,67.508362,4025258,67.168896,67.293478,66.860368,67.352372,67.215681,67.297131,-0.238395,-0.429703,0.191308,18.969639,22.407799,20.662406,68.207794,67.002510,67.901337,66.948162,67.056858,52.574323,83.244674,72.325399,-11.174433,1.071383,24.768763,-16.755326,77.777957,96.731096,68.597926,66.860368,65.122810,1.144067,67.898273,65.694257,68.361206,64.590302,-25145421,55.019508,-225494.903054,1.706651e+07,66.934523,67.750837,67.690218,0,0,0,0,0,1,1,0,0,0,1,-0.160237,-0.135584,1.614687,0.221860,1.060026,0.530007,0.160237,0.308147,0.160237,0.653275,1.294213,3.130771,1.023046,0.875135,1.503757,0.591641,-0.246525,-0.160237,0.283494,3.882649,3.007514,3.352642,3.771719,3.968937,3.118455,2.243309,2.908911,2.650071
2010-03-03,42.999077,67.725754,68.294312,67.608696,67.859535,3411351,67.324416,67.365385,66.884198,67.476833,67.308422,67.337953,-0.178915,-0.379545,0.200631,18.161231,23.029801,19.755869,65.752510,66.992058,67.901337,66.927258,67.056858,51.789581,79.403801,77.359471,-9.648542,2.298567,24.385225,-20.596199,79.800006,101.187520,68.655181,66.884198,65.113214,1.111321,67.915135,65.746237,68.361206,64.590302,-28556772,60.231547,-230961.154414,1.482003e+07,67.045667,67.876254,67.872074,0,0,1,0,0,1,0,0,0,0,0,0.024693,1.777773,0.382710,1.222222,0.691352,0.320989,0.469136,0.320989,0.814818,1.456784,3.296290,1.185182,1.037034,1.666665,0.753085,-0.086426,0.000000,0.444443,4.049375,3.172835

,adj_close,close,high,low,open,volume,sma_5,sma_10,sma_20,ema_5,ema_10,ema_20,macd,macd_signal,macd_diff,adx,adx_pos,adx_neg,parabolic_sar,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,rsi_14,stochastic_k,stochastic_d,tsi,roc_12,cmo,williams_%r,cci_10,cci_20,bollinger_upper,bollinger_middle,bollinger_lower,atr_14,keltner_hband,keltner_lband,donchian_hband,donchian_lband,obv,mfi_14,volume_oscillator,adl,vwap_14,pivot_point_1,pivot_point_2,doji,hammer,inverted_hammer,shooting_star,morning_star,triple_top,triple_bottom,red_marubozu,green_marubozu,three_black_crows,three_white_soldiers,change_1td,change_2td,change_3td,change_4td,change_5td,change_6td,change_7td,change_8td,change_9td,change_10td,change_11td,change_12td,change_13td,change_14td,change_15td,change_16td,change_17td,change_18td,change_19td,change_20td,change_21td,change_22td,change_23td,change_24td,change_25td,change_26td,change_27td,change_28td
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-02-26,0.338360,0.358992,0.337020,0.515244,0.329381,0.079047,0.338734,0.305175,0.179254,0.335000,0.280288,0.190304,0.481408,0.406917,0.629774,0.317666,0.254895,0.321448,0.504360,0.278142,0.312433,0.365534,0.334177,0.379091,0.641458,0.627388,0.239351,0.676740,0.572535,0.641458,0.452584,0.414258,0.000000,0.179254,0.491379,0.168065,0.123412,0.230228,0.021518,0.603787,0.087952,0.430364,0.495428,0.035670,0.211458,0.367879,0.362623,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.540442,0.652178,0.640060,0.619663,0.651448,0.583545,0.622201,0.580211,0.562196,0.571064,0.589737,0.632857,0.642607,0.686782,0.579264,0.605170,0.647361,0.616379,0.570254,0.551788,0.566454,0.698091,0.692523,0.726857,0.722643,0.760587,0.740946,0.685572
2010-03-01,0.367810,0.387525,0.377358,0.578760,0.384058,0.000000,0.337220,0.318044,0.179822,0.347121,0.287810,0.192586,0.497243,0.420698,0.642003,0.275747,0.342844,0.299793,0.500924,0.231421,0.312433,0.263566,0.334177,0.430937,0.692966,0.653298,0.260222,0.717370,0.693177,0.692966,0.619489,0.572413,0.001075,0.179822,0.491286,0.143194,0.115838,0.235702,0.007928,0.603787,0.120493,0.548273,0.505887,0.018164,0.233988,0.418273,0.414711,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.559590,0.581872,0.594719,0.703112,0.522780,0.601815,0.556929,0.526640,0.538865,0.531401,0.586962,0.638532,0.702696,0.572709,0.550518,0.608774,0.589043,0.559935,0.552419,0.548176,0.686739,0.644645,0.685116,0.722226,0.710545,0.711489,0.688994,0.689659
2010-03-02,0.405482,0.424021,0.405335,0.590447,0.395257,0.095478,0.358722,0.324111,0.184176,0.369319,0.303513,0.201736,0.521247,0.437638,0.668963,0.242696,0.393312,0.278341,0.497557,0.231421,0.312433,0.263566,0.334177,0.495017,0.831716,0.758532,0.290466,0.631979,0.655095,0.831716,0.697752,0.640270,0.010935,0.184176,0.489044,0.120099,0.118452,0.240426,0.007928,0.603787,0.163525,0.536235,0.523484,0.055368,0.252949,0.444785,0.437677,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.449107,0.516044,0.688900,0.560728,0.532942,0.528150,0.493081,0.495284,0.489326,0.520455,0.584096,0.702476,0.572930,0.537052,0.548038,0.543083,0.528414,0.535742,0.543038,0.661975,0.624911,0.631380,0.674963,0.704863,0.658398,0.656811,0.687829,0.656326
2010-03-03,0.396578,0.415395,0.424853,0.604675,0.422925,0.035748,0.372804,0.332016,0.187772,0.380780,0.314105,0.208340,0.538342,0.455405,0.676005,0.219411,0.416403,0.256642,0.332584,0.230055,0.312433,0.260584,0.334177,0.478266,0.793139,0.815223,0.312821,0.705691,0.651949,0.793139,0.701964,0.650598,0.018366,0.187772,0.487867,0.099408,0.120760,0.248189,0.007928,0.603787,0.127056,0.627574,0.519983,0.000000,0.266931,0.454645,0.452155,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.473350,0.707809,0.580324,0.628996,0.508630,0.515528,0.511759,0.496036,0.528275,0.566594,0.708635,0.593839,0.573576,0.570753,0.516971,0.515329,0.538216,0.560345,0.691647,0.634262,0.644855,0.653082,0.687872,0.681594,0.634631,0.685579,0.684430,0.690294
2010-03-04,0.397947,0.416722,0.404034,0.589939,0.4262